In [1]:
import os
import pysubs2
import re
import cv2
import threading

In [2]:
def Inpainting(srcImg,x,y,xx,yy,style="文本",kernelSize=7,iter=1,r=3):
    src = srcImg[x:xx,y:yy].copy()
    gray = cv2.cvtColor(src, cv2.COLOR_BGR2GRAY,1)
    if style == "学生" or style == "文本": #白字
        thresh = cv2.threshold(gray, 180, 255, cv2.THRESH_BINARY)[1]
    else:
        thresh = cv2.threshold(gray, 0, 255, cv2.THRESH_BINARY_INV | cv2.THRESH_OTSU)[1]
    kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (kernelSize, kernelSize))
    maskImg = cv2.dilate(thresh, kernel,iterations=iter)
    inpaintImg = cv2.inpaint(src,maskImg,r,cv2.INPAINT_NS)
    srcImg[x:xx,y:yy] = inpaintImg

In [6]:
# 多线程-任务
def work(start):
    video_path = os.path.join("./temp/", "part_"+str(start)+".mp4")
    cap = cv2.VideoCapture(video_path)
    fps = cap.get(cv2.CAP_PROP_FPS)
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    width = int(cap.get(3))
    height = int(cap.get(4))
    out = cv2.VideoWriter("./temp/part_"+str(start)+"_out.mp4",fourcc,fps,(width,height))
    print("start "+str(start))

    for i in range(start*num,(start+1)*num):
        ret, img = cap.read()
        if ret:
            for style in frame[i]:
                y1 ,x1 ,y2 ,x2= axis[style][0], axis[style][1], axis[style][2], axis[style][3]
                Inpainting(img,x1,y1,x2,y2,style,kernelSize=9,iter=1,r=4)
            out.write(img)
            # img = cv2.resize(img, (0, 0),fx = 0.5,fy = 0.5)
            # cv2.imshow('img',img)
        else:
            print("Error")
            break
        if cv2.waitKey(1) & 0xFF==27:
            break
    cap.release()
    out.release()
    # cv2.destroyAllWindows()

In [3]:
axis = {}
with open("config.txt", "r",encoding='utf-8') as f:
    for line in f.readlines():
        line = line.strip('\n')
        line = line.split(':')
        style = line[0]
        axis[style] = list(map(int,list(line[1].split(','))))

In [4]:
filename = "小夏4"
path = sys.path[0]+"\\temp\\"
cap = cv2.VideoCapture("小夏4.mp4")
length = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
fps = cap.get(cv2.CAP_PROP_FPS)
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
width = int(cap.get(3))
height = int(cap.get(4))
frame = []
for i in range(length):
    frame.append([])
if not os.path.exists(path):
    os.makedirs(path)

#filename_out.ass
subs = pysubs2.load(filename+".ass")
for sub in subs:
    start = pysubs2.time.ms_to_frames(sub.start, fps)
    end = pysubs2.time.ms_to_frames(sub.end, fps)

    if sub.style == "文本":
        name_school = sub.text.split("：")[0]
        text = sub.text.split("：")[-1]

        if name_school != text:
            name = name_school.split('（')[0]
            school = name_school.split('（')[-1].replace("）","")
            subs.insert(0, pysubs2.SSAEvent(start=sub.start, end=sub.end, text=name+" {\\fs35 \c&H f4ca80 }"+school, style="学生"))

        if len(text) > 35: # 换行
            text = "\\N".join(re.findall(".{35}",text)) 
        sub.text = text

        for i in range(start,end):
            frame[i].append("学生")
            frame[i].append("文本")

    else:
        for i in range(start,end):
            frame[i].append(sub.style)
subs.save(path + filename + "_out.ass")

In [ ]:
# 多线程-切分
num = 1000
i = -1

with open(path + "list.txt","a") as f:
    while 1:
        i += 1
        ret, img = cap.read()
        if ret:
            if i % num == 0:
                out = cv2.VideoWriter(path + "part_" + str(int(i/num)) + ".mp4",fourcc,fps,(width,height))
                f.write(f"file ./temp/part_{id}_out.mp4\n".format(id=int(i/num)))
            out.write(img)
            # img = cv2.resize(img, (0, 0),fx = 0.5,fy = 0.5)
            # cv2.imshow('img',img)
            #print(i)
        else:
            break
        if cv2.waitKey(1) & 0xFF==27:
            break
    cap.release()
    out.release()
    # cv2.destroyAllWindows()

In [7]:
thread_list = []

for i in range(0,8):

    thread = threading.Thread(target=work, args=[i])
    thread.start()
    thread_list.append(thread)

for t in thread_list:

    t.join()


start 0
start 1
start 6
start 5start 3
start 7
start 2start 4


Error


In [8]:
os.system("ffmpeg -f concat -safe 0 -i {path}list.txt -c copy {path}concat.mp4".format(path=path))
os.system("ffmpeg -i {path}concat.mp4 -i {filename}.mp4  -c copy -map 0 -map 1:1 -y -shortest {path}{filename}_out.mp4".format(path=path,filename="小夏4"))
os.system("ffmpeg -i {path}{filename}_out.mp4 -vf subtitles=temp/{filename}_out.ass {filename}_final.mp4".format(path=path,filename="小夏4"))
#os.system("rd/s/q "+path)

0

In [36]:
sys.path[0]

'd:\\Desktop\\code'

In [6]:
# 音频
os.system("ffmpeg -i {filename}_temp.mp4 -i {filename}.mp4  -c copy -map 0 -map 1:1 -y -shortest {filename}_out.mp4".format(filename="小夏4"))
os.system("del {filename}_temp.mp4".format(filename="小夏4"))

0

In [9]:
# 字幕https://zhuanlan.zhihu.com/p/161000532
os.system("ffmpeg -i {path}{filename}_out.mp4 -vf subtitles=temp/{filename}_out.ass {filename}_final.mp4".format(path=path,filename="小夏4"))
os.system("del {filename}_out.mp4".format(filename="小夏4"))
os.system("del {filename}_out.ass".format(filename="小夏4"))

1

In [ ]:
# 换行
subs = pysubs2.load("小夏4.ass")
print(subs[18].text)
text = "\\N".join(re.findall(".{35}",subs[18].text))
print(text)

In [23]:
work(1)

start 1


In [25]:
thread2 = threading.Thread(target=work,args = [2])
thread3 = threading.Thread(target=work,args = [3])

thread3.start()
thread2.start()

thread3.join()
thread2.join()


start 3
start 2


In [26]:
thread0 = threading.Thread(target=work,args = [0])
thread1 = threading.Thread(target=work,args = [1])
thread2 = threading.Thread(target=work,args = [2])
thread3 = threading.Thread(target=work,args = [3])
thread4 = threading.Thread(target=work,args = [4])
thread5 = threading.Thread(target=work,args = [5])
thread6 = threading.Thread(target=work,args = [6])
thread7 = threading.Thread(target=work,args = [7])

thread0.start()
thread1.start()
thread2.start()
thread3.start()
thread4.start()
thread5.start()
thread6.start()
thread7.start()

thread0.join()
thread1.join()
thread2.join()
thread3.join()
thread4.join()
thread5.join()
thread6.join()
thread7.join()

start 0
start 7
start 5start 3start 4
start 6
start 2


start 1
Error


In [12]:
from alive_progress import alive_bar
items=range(1000)# retrieve your set of items
with alive_bar(len(items)) as bar:# declare your expected total
    for item in items:# iterate as usual
        # process each item
        bar()
        # call after consuming one item

|████████████████████████████████████████| 1000/1000 [100%] in 0.0s (1075847.23/s)


In [23]:
subs = pysubs2.load("./2222/补习部2章6.ass")
for sub in subs:
    if sub.style == "文本":
        text = sub.text.split("：")[-1]
        if len(text) > 35:
            
            print(text)

说起来，前辈现在在哪里？要是您不快点下命令的话，这样下去鹤城前辈就要发射……飞奔出去了。
现在伊甸条约近在眼前，正是各种方面都很敏感的时期。如果让外界觉得，这是「圣三一的正义实现委员会与欣嫩谷之间的冲突」，不难想象，状况会变得十分不利。
也就是说，如果能以补习授业部和「Schale」一起的形式解决事件就再理想不过了。老师，能拜托您吗……？
虽然早就想到会有一天像这样一起并肩作战……看来这一天要来的稍微早一点了呢，小春。
说起来，还是第一次在老师的指挥下战斗呢。无需顾虑，老师。请尽情使用我吧。
呼呼呼，说的不错。虽然要拼一拼运气了，不过这不失为一种调味料呢！既然如此！
都，都说了不要丢下我了—！呜哇—！真是的，给我记着！总有一天我绝对会报这个仇的——！！


In [28]:
subs = pysubs2.load("./2222/补习部2章6.ass")
for sub in subs:
    if sub.style == "文本":
        text = sub.text.split("：")[-1]
        if len(text) > 35:
            text = "\\N".join(re.findall(".{35}",text)) +"\\N"+ text[-(len(text)%35):]
            print(text)

说起来，前辈现在在哪里？要是您不快点下命令的话，这样下去鹤城前辈就要发\N射……飞奔出去了。
现在伊甸条约近在眼前，正是各种方面都很敏感的时期。如果让外界觉得，这是\N「圣三一的正义实现委员会与欣嫩谷之间的冲突」，不难想象，状况会变得十分\N不利。
也就是说，如果能以补习授业部和「Schale」一起的形式解决事件就再理\N想不过了。老师，能拜托您吗……？
虽然早就想到会有一天像这样一起并肩作战……看来这一天要来的稍微早一点了\N呢，小春。
说起来，还是第一次在老师的指挥下战斗呢。无需顾虑，老师。请尽情使用我吧\N。
呼呼呼，说的不错。虽然要拼一拼运气了，不过这不失为一种调味料呢！既然如\N此！
都，都说了不要丢下我了—！呜哇—！真是的，给我记着！总有一天我绝对会报\N这个仇的——！！
